In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "00_efficientnet-b4"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 8
    model_name = 'efficientnet-b4'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  00_efficientnet-b4  already exists


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.Flip(),
        A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.ElasticTransform(p=1.0),
            A.IAAPiecewiseAffine(p=1.0)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.33, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
#     hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
#     hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b4


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True, mode="min", factor=0.3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

# end_lr = 0.1
# start_lr = 1e-7
# lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * len(train_dataloader)))
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# scheduler_1 = LinearCyclicalScheduler(optimizer, "lr", start_value=0.01, end_value=0.005, cycle_size=60)
# scheduler_2 = CosineAnnealingScheduler(optimizer, "lr", start_value=0.05, end_value=0.001, cycle_size=60)
# combined_scheduler = ConcatScheduler(schedulers=[scheduler_1, scheduler_2], durations=[30, ])

In [11]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [10]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 33s
Training Results - Average Loss: 0.8486 | auc/_mean: 0.7962 | auc/healthy: 0.8499 | auc/multiple_diseases: 0.6130, auc/rust: 0.8581, auc/scab: 0.8639
Evaluating Results - Average Loss: 0.5046 | auc/_mean: 0.9059 | auc/healthy: 0.9512 | auc/multiple_diseases: 0.7480, auc/rust: 0.9821, auc/scab: 0.9423




Epoch: 02 | Time: 0m 33s
Training Results - Average Loss: 0.6427 | auc/_mean: 0.8735 | auc/healthy: 0.9155 | auc/multiple_diseases: 0.7622, auc/rust: 0.9202, auc/scab: 0.8961
Evaluating Results - Average Loss: 0.4601 | auc/_mean: 0.9134 | auc/healthy: 0.9530 | auc/multiple_diseases: 0.7752, auc/rust: 0.9735, auc/scab: 0.9521




Epoch: 03 | Time: 0m 33s
Training Results - Average Loss: 0.5642 | auc/_mean: 0.8982 | auc/healthy: 0.9127 | auc/multiple_diseases: 0.8113, auc/rust: 0.9425, auc/scab: 0.9262
Evaluating Results - Average Loss: 0.4741 | auc/_mean: 0.8989 | auc/healthy: 0.9580 | auc/multiple_diseases: 0.7217, auc/rust: 0.9636, auc/scab: 0.9522




Epoch: 04 | Time: 0m 33s
Training Results - Average Loss: 0.5313 | auc/_mean: 0.9113 | auc/healthy: 0.9307 | auc/multiple_diseases: 0.8502, auc/rust: 0.9350, auc/scab: 0.9292
Evaluating Results - Average Loss: 0.4775 | auc/_mean: 0.8979 | auc/healthy: 0.9516 | auc/multiple_diseases: 0.7254, auc/rust: 0.9642, auc/scab: 0.9504




Epoch: 05 | Time: 0m 33s
Training Results - Average Loss: 0.5327 | auc/_mean: 0.8982 | auc/healthy: 0.9262 | auc/multiple_diseases: 0.7876, auc/rust: 0.9434, auc/scab: 0.9358
Evaluating Results - Average Loss: 0.4733 | auc/_mean: 0.8946 | auc/healthy: 0.9540 | auc/multiple_diseases: 0.7111, auc/rust: 0.9647, auc/scab: 0.9486



...


Epoch: 15 | Time: 0m 33s
Training Results - Average Loss: 0.4167 | auc/_mean: 0.9366 | auc/healthy: 0.9466 | auc/multiple_diseases: 0.8968, auc/rust: 0.9569, auc/scab: 0.9461
Evaluating Results - Average Loss: 0.4560 | auc/_mean: 0.9024 | auc/healthy: 0.9547 | auc/multiple_diseases: 0.7316, auc/rust: 0.9671, auc/scab: 0.9561




Epoch: 16 | Time: 0m 33s
Training Results - Average Loss: 0.4221 | auc/_mean: 0.9342 | auc/healthy: 0.9464 | auc/multiple_diseases: 0.8829, auc/rust: 0.9541, auc/scab: 0.9534
Evaluating Results - Average Loss: 0.4502 | auc/_mean: 0.9044 | auc/healthy: 0.9556 | auc/multiple_diseases: 0.7384, auc/rust: 0.9688, auc/scab: 0.9549




Epoch: 17 | Time: 0m 33s
Training Results - Average Loss: 0.4285 | auc/_mean: 0.9382 | auc/healthy: 0.9484 | auc/multiple_diseases: 0.9040, auc/rust: 0.9542, auc/scab: 0.9462
Evaluating Results - Average Loss: 0.4518 | auc/_mean: 0.9029 | auc/healthy: 0.9557 | auc/multiple_diseases: 0.7323, auc/rust: 0.9686, auc/scab: 0.9550

Epoch    17: reducing learning rate of group 0 to 8.1000e-05.



Epoch: 18 | Time: 0m 34s
Training Results - Average Loss: 0.4176 | auc/_mean: 0.9397 | auc/healthy: 0.9456 | auc/multiple_diseases: 0.9144, auc/rust: 0.9483, auc/scab: 0.9505
Evaluating Results - Average Loss: 0.4498 | auc/_mean: 0.9028 | auc/healthy: 0.9554 | auc/multiple_diseases: 0.7313, auc/rust: 0.9690, auc/scab: 0.9557




Epoch: 19 | Time: 0m 32s
Training Results - Average Loss: 0.4409 | auc/_mean: 0.9338 | auc/healthy: 0.9498 | auc/multiple_diseases: 0.8901, auc/rust: 0.9480, auc/scab: 0.9475
Evaluating Results - Average Loss: 0.4489 | auc/_mean: 0.9040 | auc/healthy: 0.9552 | auc/multiple_diseases: 0.7362, auc/rust: 0.9688, auc/scab: 0.9558

EarlyStopping


In [12]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=8.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config.patience, verbose=True, mode="max", factor=0.3)

Loaded pretrained weights for efficientnet-b4


In [13]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 1m 28s
Training Results - Average Loss: 1.1112 | auc/_mean: 0.7578 | auc/healthy: 0.8237 | auc/multiple_diseases: 0.5513, auc/rust: 0.8440, auc/scab: 0.8123
Evaluating Results - Average Loss: 2.1655 | auc/_mean: 0.8115 | auc/healthy: 0.9692 | auc/multiple_diseases: 0.6922, auc/rust: 0.8353, auc/scab: 0.7492




Epoch: 02 | Time: 1m 28s
Training Results - Average Loss: 0.5517 | auc/_mean: 0.8570 | auc/healthy: 0.9372 | auc/multiple_diseases: 0.5989, auc/rust: 0.9534, auc/scab: 0.9385
Evaluating Results - Average Loss: 1.0065 | auc/_mean: 0.8820 | auc/healthy: 0.9807 | auc/multiple_diseases: 0.6357, auc/rust: 0.9842, auc/scab: 0.9275




Epoch: 03 | Time: 1m 28s
Training Results - Average Loss: 0.4512 | auc/_mean: 0.8681 | auc/healthy: 0.9693 | auc/multiple_diseases: 0.5779, auc/rust: 0.9634, auc/scab: 0.9618
Evaluating Results - Average Loss: 0.5988 | auc/_mean: 0.8881 | auc/healthy: 0.9906 | auc/multiple_diseases: 0.5949, auc/rust: 0.9887, auc/scab: 0.9782




Epoch: 04 | Time: 1m 28s
Training Results - Average Loss: 0.3654 | auc/_mean: 0.8985 | auc/healthy: 0.9793 | auc/multiple_diseases: 0.6662, auc/rust: 0.9729, auc/scab: 0.9754
Evaluating Results - Average Loss: 0.2840 | auc/_mean: 0.8843 | auc/healthy: 0.9918 | auc/multiple_diseases: 0.5604, auc/rust: 0.9929, auc/scab: 0.9922




Epoch: 05 | Time: 1m 28s
Training Results - Average Loss: 0.3413 | auc/_mean: 0.9021 | auc/healthy: 0.9880 | auc/multiple_diseases: 0.6679, auc/rust: 0.9685, auc/scab: 0.9838
Evaluating Results - Average Loss: 0.5303 | auc/_mean: 0.8839 | auc/healthy: 0.9903 | auc/multiple_diseases: 0.5761, auc/rust: 0.9807, auc/scab: 0.9887



...


Epoch: 27 | Time: 1m 28s
Training Results - Average Loss: 0.0099 | auc/_mean: 0.9969 | auc/healthy: 1.0000 | auc/multiple_diseases: 0.9879, auc/rust: 0.9999, auc/scab: 0.9999
Evaluating Results - Average Loss: 0.2422 | auc/_mean: 0.9237 | auc/healthy: 0.9951 | auc/multiple_diseases: 0.7062, auc/rust: 0.9975, auc/scab: 0.9961




Epoch: 28 | Time: 1m 28s
Training Results - Average Loss: 0.0104 | auc/_mean: 0.9952 | auc/healthy: 1.0000 | auc/multiple_diseases: 0.9811, auc/rust: 1.0000, auc/scab: 0.9998
Evaluating Results - Average Loss: 0.2983 | auc/_mean: 0.9141 | auc/healthy: 0.9957 | auc/multiple_diseases: 0.6672, auc/rust: 0.9973, auc/scab: 0.9963




Epoch: 29 | Time: 1m 28s
Training Results - Average Loss: 0.0101 | auc/_mean: 0.9960 | auc/healthy: 1.0000 | auc/multiple_diseases: 0.9841, auc/rust: 1.0000, auc/scab: 0.9999
Evaluating Results - Average Loss: 0.3047 | auc/_mean: 0.9177 | auc/healthy: 0.9960 | auc/multiple_diseases: 0.6816, auc/rust: 0.9969, auc/scab: 0.9961




Epoch: 30 | Time: 1m 28s
Training Results - Average Loss: 0.0195 | auc/_mean: 0.9960 | auc/healthy: 1.0000 | auc/multiple_diseases: 0.9850, auc/rust: 0.9996, auc/scab: 0.9994
Evaluating Results - Average Loss: 0.2651 | auc/_mean: 0.9139 | auc/healthy: 0.9951 | auc/multiple_diseases: 0.6695, auc/rust: 0.9952, auc/scab: 0.9960

Epoch    30: reducing learning rate of group 0 to 2.7000e-04.



Epoch: 31 | Time: 1m 28s
Training Results - Average Loss: 0.0107 | auc/_mean: 0.9970 | auc/healthy: 1.0000 | auc/multiple_diseases: 0.9879, auc/rust: 1.0000, auc/scab: 1.0000
Evaluating Results - Average Loss: 0.2944 | auc/_mean: 0.9144 | auc/healthy: 0.9949 | auc/multiple_diseases: 0.6695, auc/rust: 0.9960, auc/scab: 0.9973

EarlyStopping


In [44]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [42]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [43]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=1_auc_mean=0.8706131482876178.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [45]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [46]:
y_probas = np.mean(result, axis=-1)

In [47]:
y_probas[:10]

array([[0.09819383, 0.3116633 , 0.1852319 , 0.40491095],
       [0.09934221, 0.13119815, 0.5209352 , 0.24852438],
       [0.00145755, 0.01327283, 0.002809  , 0.9824607 ],
       [0.9671133 , 0.00678661, 0.01244393, 0.0136562 ],
       [0.02429822, 0.02702083, 0.91830266, 0.0303783 ],
       [0.6532921 , 0.02755702, 0.12354162, 0.19560926],
       [0.8862263 , 0.03570914, 0.0465331 , 0.03153154],
       [0.0978264 , 0.04821354, 0.00504111, 0.84891903],
       [0.3122094 , 0.10618415, 0.03836362, 0.5432429 ],
       [0.27814376, 0.05621394, 0.5502282 , 0.11541412]], dtype=float32)

In [48]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.098194,0.311663,0.185232,0.404911
1,Test_1,0.099342,0.131198,0.520935,0.248524
2,Test_2,0.001458,0.013273,0.002809,0.982461
3,Test_3,0.967113,0.006787,0.012444,0.013656
4,Test_4,0.024298,0.027021,0.918303,0.030378


In [49]:
config.submission_file

'02_effb0_hard_aug.csv'